In [12]:
import plotly.express as px
import pandas as pd
from sklearn.preprocessing import StandardScaler 
from sklearn.cluster import KMeans

In [6]:
df = pd.read_csv('../data/vendas_restaurante.csv')
df.head()

,data,cliente,prato,categoria,quantidade,valor_unitario,valor_total,dia_semana,hora
0,2025-08-24 03:42:19,Isabella Gomes,Salada Caesar,Entrada,3,18.0,54.0,Sunday,3
1,2025-09-16 15:39:51,Thales Leão,Petit Gateau,Sobremesa,1,15.0,15.0,Tuesday,15
2,2025-10-02 04:54:30,Rhavi Pacheco,Strogonoff,Prato Principal,3,32.0,96.0,Thursday,4
3,2025-10-26 01:22:13,Francisco Barbosa,Salada Caesar,Entrada,3,18.0,54.0,Sunday,1
4,2025-08-10 07:24:30,Ana Beatriz Mendes,Suco Natural,Bebida,1,10.0,10.0,Sunday,7


In [7]:
fig = px.bar(
    df.groupby('prato')['valor_total'].sum().reset_index(),
    x='prato', 
    y='valor_total',
    title='Vendas por Prato'
)
fig.show()

In [11]:
clientes_df = (
    df.groupby('cliente')
    .agg({
        'valor_total': 'sum',
        'quantidade': 'sum',
        'data': ['min', 'max', 'count']
    })
)

clientes_df.columns = ['total_gasto', 'total_items', 'primeira_compra', 'ultima_compra', 'num_compras']

clientes_df['primeira_compra'] = pd.to_datetime(clientes_df['primeira_compra'])
clientes_df['ultima_compra'] = pd.to_datetime(clientes_df['ultima_compra'])
clientes_df['dias_entre'] = (clientes_df['ultima_compra'] - clientes_df['primeira_compra']).dt.days

In [14]:
X = clientes_df[['total_gasto', 'num_compras', 'dias_entre']]
X_scalled = StandardScaler().fit_transform(X)

Kmeans = KMeans(n_clusters=3, random_state=42)
clientes_df['segmento'] = Kmeans.fit_predict(X_scalled)
print(clientes_df.head(10))

                       total_gasto  total_items     primeira_compra  \
cliente                                                               
Alana da Conceição           586.0           32 2025-08-04 09:56:29   
Alexia Vasconcelos           532.0           29 2025-08-02 05:39:59   
Alice da Rosa                750.0           40 2025-08-01 00:45:29   
Ana Beatriz Mendes           281.0           18 2025-08-04 11:07:43   
Ana Carolina Dias            838.0           43 2025-08-01 23:57:29   
Ana Cecília Gonçalves        672.0           39 2025-08-06 04:34:33   
Ana Cecília Mendonça         544.0           26 2025-08-05 18:18:54   
Ana Clara Alves              452.0           30 2025-08-04 06:13:21   
Ana Julia Moraes             537.0           22 2025-08-01 09:50:44   
Ana Luiza Macedo             546.0           28 2025-08-09 23:42:42   

                            ultima_compra  num_compras  dias_entre  segmento  
cliente                                                             

In [17]:
clientes_cluster = (
    clientes_df.groupby('segmento')
    .agg({
        'total_gasto': 'mean',
        'num_compras': 'mean',
        'dias_entre': 'mean',
        'total_items': 'mean'
    })
    .round(2)
)
print(clientes_cluster)

          total_gasto  num_compras  dias_entre  total_items
segmento                                                   
0              505.35        13.75       79.27        26.69
1              783.08        18.83       80.83        38.22
2              389.00        10.21       53.86        19.79


In [20]:
fig = px.scatter(
    clientes_df, 
    x = 'num_compras', y = 'total_gasto',
    color = clientes_df['segmento'].astype(str),
    title = 'Segmentação de Clientes - Restaurante Sabor & Arte',
    labels = {'num_compras': 'Nº de Compras', 'total_gasto': 'Total Gasto (R$)', 'color': 'Segmento'}
)
fig.show()

In [22]:
# Salvar para usar no dashboard
clientes_df.to_csv('../data/clientes_segmentados.csv', index=True)